Ressource : [https://rruntsch.medium.com/how-to-write-a-python-program-to-query-biomedical-journal-citations-in-the-pubmed-database-c7e842e4df89](https://rruntsch.medium.com/how-to-write-a-python-program-to-query-biomedical-journal-citations-in-the-pubmed-database-c7e842e4df89) 

## Oumaima

In [ ]:
import json # PubMed database searches retrieve results in JSON format. This module provides functions to access data in JSON structures.
import urllib.request # The urlopen() function sends the request URL to the NCBI server to execute and retrieve search results and citations.
import time

In [ ]:
def retrieve_abstract():       
    urls = []
    base_url_efetch = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?"
    output_file_path = r"/pubmed_xml_fies/"
    sleep_minutes = .2
        
    ids = ['13n'+str(n).zfill(4) for n in range(1,3)]
    for id in ids:
        url = base_url_efetch + 'db=pubmed&id=' + id + '&retmode=text&retmax=text&rettype=abstract'
        urls.append(url)
        result = urllib.request.urlopen(url)
        text = result.read().decode('utf-8')
        file_name = output_file_path + "pubmed_" + id + ".txt"
        
        file_out = open(file_name, "w", encoding="utf-8")
        file_out.write(text)
        file_out.close()
        
        # Sleep so that pubmed is not overloaded
        time.sleep(sleep_minutes)

## Tanish

In [ ]:
retrieve_abstract()

In [ ]:
!pip install git+git://github.com/titipata/pubmed_parser.git #This needs to be run in order for the pubmed_parser to work

  Cloning git://github.com/titipata/pubmed_parser.git to /tmp/pip-req-build-h2cd5vfi
  Running command git clone -q git://github.com/titipata/pubmed_parser.git /tmp/pip-req-build-h2cd5vfi
     |████████████████████████████████| 245kB 8.0MB/s 
     |████████████████████████████████| 245kB 13.4MB/s 
  Created wheel for pubmed-parser: filename=pubmed_parser-0.2.2-cp37-none-any.whl size=18334 sha256=9692a29dfe8cd95416293c53d7f6127d0a29cc1b7c4d6f48b856bd1778a95982
  Stored in directory: /tmp/pip-ephem-wheel-cache-8uudkwm1/wheels/f5/4c/84/40073501015d7126a260f0cdae4ebae8a37e8e480ee36ff9ce
Successfully built pubmed-parser
ERROR: datascience 0.10.6 has requirement coverage==3.7.1, but you'll have coverage 5.5 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: coveralls 0.5 has requirement coverage<3.999,>=3.6, but you'll have coverage 5.5 which is incompatible.
ERROR: pytest-cov 2.12.1 has requirement pytes

In [ ]:
import os
import requests
import pubmed_parser as pp
import pandas as pd

In [ ]:
url_list = []
numbers_in_file_name = range(1, 3) #1062 is max so use 1063 if attempting to access the last file. CAUTION: pulling too many .xml files probably will cause this to crash. 
total_dataframe = pd.DataFrame() #Empty dataframe which will eventually contain all data parsed using the PubMed parser

for n in numbers_in_file_name:
  url_list.append("https://ftp.ncbi.nlm.nih.gov/pubmed/baseline/pubmed21n" + str(n).zfill(4) + ".xml.gz") #Creating relevant links to pull data. 

for n in url_list:
  print(n) #Printing those links to double-check. 

for url in url_list:
  r = requests.get(url, stream = True) 

  with open(url[45:62],"wb") as xml:
    for chunk in r.iter_content(chunk_size=1024):
        if chunk:
            xml.write(chunk) #Pulling Data and writing it to the working directory (which in this case is "\content"). 
  raw_data = pp.parse_medline_xml("/content/" + url[45:62]) #Parsing data
  #print(raw_data[1:3]) #Can be used to check raw_data
  initial_list = list(raw_data) #Making sure the data is a list and putting values into a new variable. 
  new_df = pd.DataFrame(initial_list) #Creating dataframe with the batch of parsed data.
  total_dataframe = total_dataframe.append(new_df, ignore_index = True) #Appending the dataframe created above to the total dataframe containing all parsed data. 
    

print("\n", total_dataframe.head(100), "\n") #Double-checking to make sure the data came out right. 


pruned_dataframe = total_dataframe[['pmid','pubdate','abstract']] #Contains only the relevant columns that we need for the Stanford Parser. 
pruned_dataframe['pmid'] = pruned_dataframe['pmid'].astype(int) #Ignore the error concerning the SettingWithCopyWarning. I can't seem to get around it and the data type for the two columns does seem to change anyways. 
pruned_dataframe['pubdate'] = pruned_dataframe['pubdate'].astype(int) #

print(pruned_dataframe.head(100), "\n") #Double-checking data
print(pruned_dataframe.dtypes, "\n") #Double-checking column data types 

https://ftp.ncbi.nlm.nih.gov/pubmed/baseline/pubmed21n0001.xml.gz
https://ftp.ncbi.nlm.nih.gov/pubmed/baseline/pubmed21n0002.xml.gz

                                                 title  ...              country
0   Formate assay in body fluids: application in m...  ...        United States
1   Delineation of the intimate details of the bac...  ...        United States
2   Metal substitutions incarbonic anhydrase: a ha...  ...        United States
3   Effect of chloroquine on cultured fibroblasts:...  ...        United States
4   Atomic models for the polypeptide backbones of...  ...        United States
..                                                ...  ...                  ...
95  Hydrogen peroxide and iron: a microbial cellul...  ...        United States
96  [Luminescence study of the effect of temperatu...  ...  Russia (Federation)
97                    Microbial sources of cellulase.  ...        United States
98   Mannosidosis: clinical and biochemical findings.  ...        

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
